In [35]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from nilmtk import DataSet
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import defaultdict
from joblib import Memory
import pickle
from pprint import pprint

#from nilmtk import DataSet
import multiprocessing as mp

from typing import Dict

!pip install -q scikit-learn-intelex
#from sklearnex import patch_sklearn
#patch_sklearn()

from sklearn import pipeline, metrics, linear_model, model_selection, multioutput, tree, ensemble, neural_network
!pip install xgboost -q
import xgboost as xgb

import numpy as np
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import LearningRateScheduler

import numpy as np
#import graphviz
import keras
from sklearn.utils import class_weight
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
#from ann_visualizer.visualize import ann_viz
from matplotlib import pyplot as plt 
from sklearn.metrics import multilabel_confusion_matrix, classification_report, confusion_matrix
# GPU goes brrrrrrrrrrrrrrrrrrrr
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

!pip3 install tqdm -q
from tqdm import tqdm
import NUK

In [69]:
HolyDataset_UKD = pickle.load(open('./datasets/HolyDatasetDALE.pkl', 'rb'))
HolyDataset_REF = pickle.load(open('./datasets/HolyDataset.pkl', 'rb'))

In [70]:
# Parameters

epochs = 20
window_size = 2550
batch_size = 128
NmDevices = 5
# table_of_options = [1,2,3,4]
k = 1
# Define a learning rate scheduler function
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

RANDOM_SEED = 42
test_size = 0.2


In [71]:

x_UKD, y_UKD, labels_UKD = HolyDataset_UKD[0], HolyDataset_UKD[2], HolyDataset_UKD[4]
class_weights_UKD = NUK.class_weights_tool(y_UKD)

x_REF, y_REF, labels_REF = HolyDataset_REF[0], HolyDataset_REF[2], HolyDataset_REF[4]
class_weights_REF = NUK.class_weights_tool(y_REF)


# For UKD dataset
x_train_UKD, x_test_UKD, y_train_UKD, y_test_UKD = train_test_split(x_UKD, y_UKD, test_size=test_size, random_state=RANDOM_SEED)

# For REF dataset
x_train_REF, x_test_REF, y_train_REF, y_test_REF = train_test_split(x_REF, y_REF, test_size=test_size, random_state=RANDOM_SEED)


In [72]:

"""
Train on UKD, evaluate on REF
"""
evaluation_results_UKD_UKD, evaluation_results_UKD_REF, evaluation_results_REF_UKD, evaluation_results_REF_REF = [], [], [], []


model = NUK.PC0(NmDevices, window_size, 'gru', 128, k)
model.build((len(y_train_UKD) + len(y_test_UKD), window_size, 1))
#model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003), loss='binary_crossentropy', metrics=[NUK.F1Score, NUK.WeightedF1Score(NUK.class_weights_tool(y_test_UKD))])
lr_scheduler = LearningRateScheduler(scheduler)
model.fit(x_train_UKD, y_train_UKD, batch_size=batch_size, epochs=epochs, class_weight=class_weights_UKD, callbacks=[lr_scheduler])

# Evaluate the model on the UKD dataset
y_pred_UKD = model.predict(x_test_UKD)
y_pred_tf_UKD = (y_pred_UKD > 0.5)
report_UKD = metrics.classification_report(y_test_UKD, y_pred_tf_UKD, target_names=labels_UKD, zero_division=0)
# I prefer to save class. reports in pandas dataframes so i use this function
report_UKD_DF = NUK.ClassificationReportToDF(report_UKD, labels_UKD)
report_UKD_DF.rename(columns={report_UKD_DF.columns[0]: "device"}, inplace=True)
evaluation_results_UKD_UKD.append(report_UKD_DF)

# Evaluate the model on the REF dataset
y_pred_REF = model.predict(x_REF)
y_pred_tf_REF = (y_pred_REF > 0.5)
report_REF = metrics.classification_report(y_REF, y_pred_tf_REF, target_names=labels_REF, zero_division=0)
# I prefer to save class. reports in pandas dataframes so i use this function
report_REF_DF = NUK.ClassificationReportToDF(report_REF, labels_REF)
report_REF_DF.rename(columns={report_REF_DF.columns[0]: "device"}, inplace = True)
evaluation_results_UKD_REF.append(report_REF_DF)


"""
Train on REF, evaluate on UKD
"""

# x_train_REF, x_val_REF = x_REF[train_idx], x_REF[val_idx]  # Swap 'x_train_UKD' with 'x_train_REF' and 'x_val_UKD' with 'x_val_REF'
# y_train_REF, y_val_REF = y_REF[train_idx], y_REF[val_idx]  # Swap 'y_train_UKD' with 'y_train_REF' and 'y_val_UKD' with 'y_val_REF'


model = NUK.PC0(NmDevices, window_size, 'gru', 128, k)
model.build((len(y_train_REF) + len(y_test_REF), window_size, 1))  # Swap 'y_train_UKD' with 'y_train_REF'
#model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003), loss='binary_crossentropy', metrics=[NUK.F1Score, NUK.WeightedF1Score(NUK.class_weights_tool(y_test_REF))])  # Swap 'y_test_UKD' with 'y_test_REF'
lr_scheduler = LearningRateScheduler(scheduler)
model.fit(x_train_REF, y_train_REF, batch_size=batch_size, epochs=epochs, class_weight=class_weights_REF, callbacks=[lr_scheduler])  # Swap 'x_train_UKD' with 'x_train_REF' and 'y_train_UKD' with 'y_train_REF'

# Evaluate the model on the REF dataset  # Swap 'UKD' with 'REF'
y_pred_REF = model.predict(x_test_REF)  # Swap 'x_val_UKD' with 'x_val_REF'
y_pred_tf_REF = (y_pred_REF > 0.5)
report_REF = metrics.classification_report(y_test_REF, y_pred_tf_REF, target_names=labels_REF, zero_division=0)  # Swap 'y_val_UKD' with 'y_val_REF'
# I prefer to save class. reports in pandas dataframes so i use this function
report_REF_DF = NUK.ClassificationReportToDF(report_REF, labels_REF)
report_REF_DF.rename(columns={report_REF_DF.columns[0]: "device"}, inplace=True)
evaluation_results_REF_REF.append(report_REF_DF)  # Swap 'evaluation_results_UKD_REF' with 'evaluation_results_REF_REF'

# Evaluate the model on the UKD dataset  # Swap 'REF' with 'UKD'
y_pred_UKD = model.predict(x_UKD)  # Swap 'x_REF' with 'x_UKD'
y_pred_tf_UKD = (y_pred_UKD > 0.5)
report_UKD = metrics.classification_report(y_UKD, y_pred_tf_UKD, target_names=labels_UKD, zero_division=0)  # Swap 'y_REF' with 'y_UKD'
# I prefer to save class. reports in pandas dataframes so i use this function
report_UKD_DF = NUK.ClassificationReportToDF(report_UKD, labels_UKD)
report_UKD_DF.rename(columns={report_UKD_DF.columns[0]: "device"}, inplace=True)
evaluation_results_REF_UKD.append(report_UKD_DF)  # Swap 'evaluation_results_UKD_UKD' with 'evaluation_results_REF_UKD'




print("\n****************************** UKD_UKD:\n",evaluation_results_UKD_UKD,"\n************************ UKD_REF: \n", evaluation_results_UKD_REF,"\n******************************REF_UKD:\n", evaluation_results_REF_UKD,"\n******************************REF_REF:\n", evaluation_results_REF_REF)


GRU enabled
Epoch 1/50
600/600 [==============================] - 31s 46ms/step - loss: inf - F1Score: 0.6649 - WeightedF1: 0.6981 - lr: 3.0000e-04
Epoch 2/50
600/600 [==============================] - 28s 47ms/step - loss: inf - F1Score: 0.7754 - WeightedF1: 0.7937 - lr: 3.0000e-04
Epoch 3/50
600/600 [==============================] - 28s 47ms/step - loss: inf - F1Score: 0.8135 - WeightedF1: 0.8289 - lr: 3.0000e-04
Epoch 4/50
600/600 [==============================] - 28s 46ms/step - loss: inf - F1Score: 0.8493 - WeightedF1: 0.8641 - lr: 3.0000e-04
Epoch 5/50
600/600 [==============================] - 28s 46ms/step - loss: inf - F1Score: 0.8705 - WeightedF1: 0.8848 - lr: 3.0000e-04
Epoch 6/50
600/600 [==============================] - 28s 46ms/step - loss: inf - F1Score: 0.8818 - WeightedF1: 0.8955 - lr: 3.0000e-04
Epoch 7/50
600/600 [==============================] - 28s 46ms/step - loss: inf - F1Score: 0.8914 - WeightedF1: 0.9037 - lr: 3.0000e-04
Epoch 8/50
251/600 [===========>....

KeyboardInterrupt: 